# Embeddings

Привет! Сегодня ты поработаешь с эмбеддингами: сделаешь классификатор эмоции твитов. Для начала, загрузи их:

In [1]:
%%capture 
%%bash

gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download

additional packages:

установим дополнительные пакеты: для замены contractions и замены эмотиконов(предварительно я проверил на данном датасете, их немало) на слова, отражающие их смысл, что может в случае анализа эмоций положительно повлиять на модель

In [2]:
%%capture 
%%bash

pip install contractions # replacing contractions - I've > I have 
pip install git+https://github.com/NeelShah18/emot # replacing emoticons - ;) > wink
pip install optuna
pip install transformers

In [3]:
import contractions
import emot

Заимпортируй библиотеки и сделай работу скриптов вопсроизводимой.

In [4]:
import math
import random
import string
import gc
import numpy as np
import pandas as pd
import seaborn as sns
import regex as re
import torch
import gensim
import gensim.downloader as api
from html import unescape
from collections import defaultdict
from typing import Dict
from sklearn.feature_extraction.text import TfidfVectorizer

from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, random_split
import torch.nn as nn
from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from transformers import get_linear_schedule_with_warmup
from torch.nn import functional as F
torch.__version__

'1.6.0+cu101'

In [5]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

отключаем warnings

In [7]:
import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

время выполнения в colab

In [8]:
%%capture
!pip install ipython-autotime -qq
%load_ext autotime

time: 237 µs


In [9]:
data = pd.read_csv("archive.zip", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

time: 4.94 s


Посмотрим на данные

In [10]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


time: 32.2 ms


функции препроцессинга:

In [11]:
def fix_emoticon(s):
  """
  replacing emoticons by related meaning word
  example: ";)" - ":wink:" 
  """
  d = emot.emoticons(s)
  if not d.get("flag", False): return s
  idx=0
  r = ""
  for meaning, location in zip(d["mean"], d["location"]):
    start, end = location
    r+=s[idx:start]+":"+meaning.split()[0].lower()+":"
    idx=end
  del d
  return r

time: 6.1 ms


In [12]:
def clean_column(col:pd.Series) -> pd.Series:
  #fixing html entities
  col = np.vectorize(unescape)(col) 
  col = pd.Series(np.vectorize(contractions.fix)(col)) 
  # removing urls
  col = col.replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True) 
  col = pd.Series(np.vectorize(fix_emoticon)(col))
  #user
  col = col.str.replace('@[A-Za-z0-9]+', '') 
  col = col.str.strip()
  col = col.str.strip(string.punctuation)
  #repeated spaces
  col = col.replace(r'\s{2,}', ' ')  
  col = col.str.lower()
  col = pd.Series(np.vectorize(lambda s: " ".join(t for t in s.split() if t not in stop_words))(col))
  return col

time: 9.41 ms


In [13]:
data["text"] = clean_column(data["text"])

time: 2min 18s


In [14]:
examples = data["text"].sample(10)
print("\n".join(examples))

ahhh hope ok
cool , tweet apps razr 2
know family drama. lame.hey next time hang kim n guys like sleepover whatever, ill call
school email open geography stuff revise! *stupid school* :crying
upper airways problem
going miss pastor's sermon faith
lunch....dj come eat
oh feeling like
gahh noo!peyton needs live!this horrible
thank glad like it! product review bit site enjoy knitting
time: 46.2 ms


Текст очень грязные. Надо добавить очистку текста в его предобработку. 

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [15]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

time: 1.05 s


## Обработка текста

Стокенизируем текст, избавим от знаков пунктуации и мелких слов.

In [16]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

nelson north devon coast next weeks devon may sometime hope though
time: 2.3 ms


In [17]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

nelson north devon coast next weeks devon sometime hope though
time: 1.97 ms


Загрузим предобученную модель эмбеддингов. Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data . 

In [18]:
word2vec = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded
time: 7min 11s


In [19]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)
time: 2.48 ms


Эмбеддинги не нормализированны, поэтому это тоже надо делать (нейросети и не только любят нормальные данные).

In [20]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False, False, False, False, False, False, False, False]
time: 3.52 s


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [21]:
class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        self.data = data
        self.feature_column = feature_column
        self.target_column = target_column
        self.word2vec = word2vec
        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])
        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)
        return {"feature": embeddings, "target": label, "text": text}

    def get_tokens_(self, text):
        return [w for w in self.tokenizer.tokenize(text) if not any(c in string.punctuation for c in w) and len(w) > 3]

    def get_embeddings_(self, tokens):
        embeddings = [(self.word2vec.get_vector(token) - self.mean) / self.std for token in tokens if token in word2vec and len(token) > 3]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings


    def __len__(self):
        return self.data.shape[0]

time: 30.2 ms


In [22]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

time: 3.01 s


## Average embedding
---
Попробуем получить векторное представление предложения из эмбеддингов слов. Самый простой вариант: усреднить вектора по всем словам. Полученный вектор можно отправить любому классификатору как вектор признаков.

Посмотрим, насколько хорошо усреднее работает для определение эмоций твитов. Сделаем их визуализацию.

In [23]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280
time: 232 ms


Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Можно получить более аккуратными алгоритмами, но данный алгоритм покажет сложность задачи и поможет оценить требования к классификатору.

In [24]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(examples["features"])

time: 47 ms


In [25]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

time: 348 ms


In [26]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1002', ...)

time: 55.3 ms


Скорее всего, ты увидел не сильно различающиеся классы. Возможно, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую(хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


> Да так и есть

In [30]:
batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

time: 128 ms


Для обученния и тестирования нейросетевой модели сделаем отдельные функции.

In [34]:
def accuracy(outputs, labels):
    predicted = torch.argmax(outputs, dim=1)
    return torch.mean(torch.eq(predicted, labels).float()).item()

def training(model, optimizer, criterion, train_loader, epoch, device="cpu", scheduler=None):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for i, batch in enumerate(pbar):
        features = batch["features"].to(device)
        targets = batch["targets"].float().unsqueeze(1).to(device)

        model.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        if scheduler is not None: scheduler.step()
        optimizer.zero_grad()

        acc = accuracy(outputs, targets)

        pbar.set_description(f"Epoch:{e + 1}.Train Loss:{loss:.4} Acc:{acc:.4}")
        
        if i%100==0:gc.collect();torch.cuda.empty_cache();
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].float().to(device)

            outputs = model(features)
            loss = criterion(outputs, targets)
            acc = accuracy(outputs, targets)

            mean_loss += loss.item()
            mean_acc += acc

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

time: 40.8 ms


Создадим модель, оптимизатор и целевую функцию. Можете создавать любую модель, с любым количеством слоев, функций активации и прочее.


Обучим модель и протестируем её.
После каждой эпохи, проверим качество модели по валидационной части датасета. Если метрика стала лучше, будем сохранять модель. Подумай, какая метрика (точность или лосс) будет лучше работать в этой задаче? 

Здесь и далее реализованно с помощью лосс. Если думаешь, что лучше сравнивать модель через качество, то поменяй код выбора модели.

### Linear

In [32]:
class LinearModel(nn.Module):

    def __init__(self, vector_size, num_classes):
        super(LinearModel, self).__init__()
        self.fc1 = nn.Linear(vector_size, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, X):
        out = F.relu(self.fc1(X))
        out = F.relu(self.fc2(out))
        return out

time: 4.1 ms


In [35]:
vector_size = dev.word2vec.vector_size
num_classes = 1
lr = 1e-2
num_epochs = 2

model = LinearModel(vector_size=vector_size, num_classes=num_classes)
model = model.cuda()

criterion = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters())
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                        num_warmup_steps=0,
                                        num_training_steps=total_steps)

best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device, scheduler)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.16390105998516083, 'Test Acc': 0.49966796875}



{'Test Loss': 0.16726750802993776, 'Test Acc': 0.49966796875}
time: 3min 4s


### LSTM


In [39]:
class ClassifierLSTM(nn.Module):
    
    def __init__(self, embedding_dim, n_hidden, n_output, n_layers, drop_p = 0.1):
        super(ClassifierLSTM, self).__init__()
        
        self.n_layers = n_layers   
        self.n_hidden = n_hidden   
        self.embedding_dim = embedding_dim
        self.lstm = nn.LSTM(self.embedding_dim, n_hidden, n_layers, batch_first = True, dropout = drop_p)
        self.classifier = nn.Sequential( nn.Dropout(drop_p),
                                         nn.Linear(n_hidden, 1),
                                         nn.Sigmoid() )
        
        
    def forward (self, inputs):
        out, _ = self.lstm(inputs.unsqueeze(1), None)
        out = out[:, -1, :] 
        out = self.classifier(out)
        return out

time: 9.79 ms


In [40]:
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 5e-5
num_epochs = 5

model = ClassifierLSTM(vector_size, n_hidden=512, n_output=1, n_layers=2)
model = model.cuda()
criterion = nn.BCELoss()
total_steps = len(train_loader) * num_epochs
optimizer = AdamW(model.parameters(),lr=5e-5,eps=1e-8)
        
scheduler = CosineAnnealingLR(optimizer, T_max=100)
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device, scheduler=scheduler)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.5527697100639343, 'Test Acc': 0.49966796875}



{'Test Loss': 0.545788709640503, 'Test Acc': 0.49966796875}



{'Test Loss': 0.5413245649337769, 'Test Acc': 0.49966796875}



{'Test Loss': 0.5381483082771301, 'Test Acc': 0.49966796875}



{'Test Loss': 0.5336405010223388, 'Test Acc': 0.49966796875}
time: 7min 55s


In [41]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.5325732929066728, 'Test Acc': 0.49990015974440893}
time: 26 s


## TF-iDF
---

Вместо обычного усреднения эмбеддингов их можно дополнительно перевзвесить. Для этого воспользуемся алгоритмом `TD-iDF`. Он уже реализован в библиотеке `scikit-learn`, остается только его добавить в наш пайплайн.

In [50]:
class TwitterDatasetTfIdf(TwitterDataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec, weights: Dict[str, float] = None):
        super().__init__(data, feature_column, target_column, word2vec)

        if weights is None:
            self.weights = self.get_tf_idf_()
        else:
            self.weights = weights

    def get_embeddings_(self, tokens):
        embeddings = [(self.word2vec.get_vector(token) - self.mean) / self.std  * self.weights.get(token, 1) for token in tokens if token in word2vec and len(token) > 3]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def get_tf_idf_(self):
        # Надо обучить tfidf на очищенном тексте. Но он принимает только список текстов, а не список списка токенов. Надо превратить второе в первое
        tokenized_texts = self.data["text"].tolist()
        tf_idf = TfidfVectorizer()
        tf_idf.fit(tokenized_texts)
        return dict(zip(tf_idf.get_feature_names(), tf_idf.idf_))


time: 17.2 ms


In [51]:
dev = TwitterDatasetTfIdf(dev_data, "text", "emotion", word2vec)

time: 15.7 s


Посмотрим на сложность получившейся задачи используя визуализацию через `PCA`.

In [52]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280
time: 314 ms


In [53]:
pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(examples["features"])

time: 23.5 ms


In [54]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1108', ...)

time: 52.9 ms


Создать нейросетку, обучим её на этих данных.

In [57]:
train_size = math.ceil(len(dev) * 0.99)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

time: 112 ms


### Linear

In [58]:
vector_size = dev.word2vec.vector_size
num_classes = 1
lr = 1e-2
num_epochs = 2

model = LinearModel(vector_size=vector_size, num_classes=num_classes)
model = model.cuda()

criterion = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters())
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                        num_warmup_steps=0,
                                        num_training_steps=total_steps)

best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device, scheduler)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.2496310552725425, 'Test Acc': 0.4960186298076923}



{'Test Loss': 0.24961109230151543, 'Test Acc': 0.4960186298076923}
time: 3min 26s


In [60]:
test = TwitterDatasetTfIdf(test_data, "text", "emotion", word2vec, weights=dev.weights)

test_loader = DataLoader(
    test, 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.24758679698260067, 'Test Acc': 0.49990015974440893}
time: 28.3 s


### LSTM

In [61]:
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 5e-5
num_epochs = 5

model = ClassifierLSTM(vector_size, n_hidden=512, n_output=1, n_layers=2)
model = model.cuda()
criterion = nn.BCELoss()
total_steps = len(train_loader) * num_epochs
optimizer = AdamW(model.parameters(),lr=5e-5,eps=1e-8)
        
scheduler = CosineAnnealingLR(optimizer, T_max=100)
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device, scheduler=scheduler)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.546238151880411, 'Test Acc': 0.4960186298076923}



{'Test Loss': 0.5349873441916245, 'Test Acc': 0.4960186298076923}



{'Test Loss': 0.5290381266520574, 'Test Acc': 0.4960186298076923}



{'Test Loss': 0.5251111319431891, 'Test Acc': 0.4960186298076923}



{'Test Loss': 0.5244966080555549, 'Test Acc': 0.4960186298076923}
time: 8min 41s


In [62]:
test = TwitterDatasetTfIdf(test_data, "text", "emotion", word2vec, weights=dev.weights)

test_loader = DataLoader(
    test, 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


{'Test Loss': 0.5202271925946014, 'Test Acc': 0.49990015974440893}
time: 28.2 s


Есть ли разница в качестве между способами? Получилось ли улучшить качество модели?


----

----


<div id="markdownResult" style="background-color: rgb(255, 255, 255); margin: auto; width: 477px;"><table><tbody><tr><th align="center"></th><th align="center"><b>Train Acc</b></th><th align="center"><b>Train Loss</b></th><th align="center"><b>Test Acc</b></th><th align="center"><b>Test Loss</b></th></tr><tr><td align="center">Linear + Avg</td><td align="center">0.506</td><td align="center">0.088</td><td align="center">0.499</td><td align="center">0.167</td></tr>
<tr><td align="center">LSTM + Avg</td><td align="center">0.502</td><td align="center">0.495</td><td align="center">0.499</td><td align="center">0.533</td></tr>
<tr><td align="center">Linear + TfIdf</td><td align="center">0.472</td><td align="center">0.26</td><td align="center">0.499</td><td align="center">0.247</td></tr>
<tr><td align="center">LSTM + TfIdf</td><td align="center">0.497</td><td align="center">0.555</td><td align="center">0.499</td><td align="center">0.52</td></tr>
</tbody></table></div>

как видим результаты не очень( что подозрительно accuracy везде одинаковый на тест сете(валид сет результаты в таблице не указаны)

ну разница как видим не большая. собсственно и по PCA также было видно  ,что классы сложно различимы. дальнейшие мысли - поменять эмбеддинги и попробовать CNN и finetuning Bert модели

Сделай небольшое исследование:
- Попробуй сделать несколько нейросеток в качестве классификатора
- Попробуй другие предобученные эмбеддинги
- Попробуй очистить текст от ников ("@username"), url-ов и других символов

Для реализации последнего тебе могут помочь регулярные выражения (`import re`). Напише ниже отчет, что ты попробовал и что получилось.

---
были использованы обычная линейная модель , LSTM (также сделал CNN, но результат был плохой, удалил)
тексты были почищены - от урлов, от ников, стоп слов, эмотиконы преобразованы в соответствующие слова по смыслу, были преобразованы contractions
что технологически не совсем верно было сделано , сравнение под разными параметрами и эпохами, но как видим примерно один везде и тот же результат
что касается метрик, я бы смотрел по лоссу и взвешенную метрику f1-score. по лоссу можно было вероятно посмотреть сами данные (отсортировать от большего лосса к меньшему) и посмотреть где происходит падение(большой лосс)

---